In [3]:
! pip install newsapi

In [22]:
import requests
import newsapi_credentials

params = {
    'q': 'corona virus',
    #'source': 'bbc-news',
    'sortBy': 'top',
    'language': 'en',
    #'category': 'business',
    #'country': 'us',
    #'apiKey': API_KEY,
}

headers = {
    'X-Api-Key': API_KEY,  # KEY in header to hide it from url
}

url = 'https://newsapi.org/v2/top-headlines'

response = requests.get(url, params=params, headers=headers)
data = response.json()

articles = data["articles"] 
results = [arr["title"] for arr in articles] 

for i, arr in enumerate(results, 1): 
    print(i, arr)

1 Trump signs executive actions on economic impact of coronavirus
2 8/9: O'Brien, Lucas, Lamont, Gottlieb, Evans, Salvanto
3 The overnight coronavirus expert
4 CNN anchor to Trump supporter: What you're saying is unscientific - CNN Video
5 Vic Premier Daniel Andrews says drop in virus cases ‘promising’, warns of more deaths
6 Coronavirus update: WHO chief calls for tenfold increase in funding for COVID-19 treatments, Red Cross goes to work in North Korea
7 Trump says Schumer, Pelosi 'want to meet to make a deal' on coronavirus relief
8 Coronavirus live updates: 97,000 kids tested positive the last two weeks of July; college football faces decision time on fall season
9 Coronavirus Australia: Depressing picture of country’s future
10 How Texas went from a coronavirus slow burn to an explosive disaster
11 Coronavirus Live Updates: Nearly 100,000 U.S. Children Tested Positive For COVID-19 In 2 Weeks
12 NI records 76 cases of Covid-19, one death
13 Coronavirus live updates: 97,000 kids tes

In [23]:
import transformers
import torch
from torch import nn
from transformers import BertModel

In [24]:
class SentimentClassifier(nn.Module):
    
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [25]:
MAX_LEN=160
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
class_names = ['negative', 'neutral', 'positive']
device='cuda'

In [26]:
model = SentimentClassifier(len(class_names))
model.load_state_dict(torch.load('model_1_2-Copy1.pth'))
model = model.to(device)

In [27]:
tokenizer = transformers.BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [28]:
for text in results[:10]:

    encoded_review = tokenizer.encode_plus(
        text,
        max_length=MAX_LEN,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    
    print(f'Review text: {text}')
    print(f'Sentiment : {class_names[prediction]}')
    print('-'*10)

Review text: Trump signs executive actions on economic impact of coronavirus
Sentiment : negative
----------
Review text: 8/9: O'Brien, Lucas, Lamont, Gottlieb, Evans, Salvanto
Sentiment : positive
----------
Review text: The overnight coronavirus expert
Sentiment : positive
----------
Review text: CNN anchor to Trump supporter: What you're saying is unscientific - CNN Video
Sentiment : negative
----------
Review text: Vic Premier Daniel Andrews says drop in virus cases ‘promising’, warns of more deaths
Sentiment : neutral
----------
Review text: Coronavirus update: WHO chief calls for tenfold increase in funding for COVID-19 treatments, Red Cross goes to work in North Korea
Sentiment : negative
----------
Review text: Trump says Schumer, Pelosi 'want to meet to make a deal' on coronavirus relief
Sentiment : neutral
----------
Review text: Coronavirus live updates: 97,000 kids tested positive the last two weeks of July; college football faces decision time on fall season
Sentiment : ne